In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# -------------------- PARAMETERS --------------------
IMG_HEIGHT, IMG_WIDTH = 128, 128
base_path = "/kaggle/input/brain-tumor-segmentation"
image_dir = os.path.join(base_path, "images")
mask_dir = os.path.join(base_path, "masks")

# -------------------- DICE COEFFICIENT & LOSS --------------------
def dice_coefficient(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1e-7) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1e-7)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def combo_loss(y_true, y_pred):
    bce = BinaryCrossentropy()(y_true, y_pred)
    d_loss = dice_loss(y_true, y_pred)
    return bce + d_loss

# -------------------- CBAM MODULE --------------------
def cbam_block(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = layers.Dense(channel // ratio, activation='relu')
    shared_layer_two = layers.Dense(channel)

    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)
    cbam_feature = layers.Reshape((1, 1, channel))(cbam_feature)
    cbam_feature = layers.Multiply()([input_feature, cbam_feature])

    return cbam_feature

# -------------------- UNET WITH VGG16 ENCODER + CBAM --------------------
def build_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_size)
    inputs = base_model.input
    skip_connections = [
        base_model.get_layer('block1_conv2').output,
        base_model.get_layer('block2_conv2').output,
        base_model.get_layer('block3_conv3').output,
        base_model.get_layer('block4_conv3').output
    ]
    x = base_model.get_layer('block5_conv3').output

    filters = [512, 256, 128, 64]
    for i in range(4):
        x = layers.UpSampling2D((2, 2))(x)
        skip = skip_connections[3 - i]
        if x.shape[1] != skip.shape[1] or x.shape[2] != skip.shape[2]:
            skip = layers.UpSampling2D(size=(2, 2))(skip)
        x = layers.Concatenate()([x, skip])
        x = cbam_block(x)
        x = layers.Conv2D(filters[i], 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(filters[i], 3, activation='relu', padding='same')(x)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss=combo_loss, metrics=['accuracy', MeanIoU(num_classes=2), dice_coefficient])
    return model

# -------------------- LOAD AND PREPARE DATA --------------------
def load_data():
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))
    images, masks = [], []
    for img_name, mask_name in zip(image_filenames, mask_filenames):
        img = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, img_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
        mask = tf.keras.preprocessing.image.load_img(os.path.join(mask_dir, mask_name), color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
        images.append(tf.keras.preprocessing.image.img_to_array(img) / 255.0)
        masks.append(tf.keras.preprocessing.image.img_to_array(mask) / 255.0)
    return np.array(images), np.array(masks)

# -------------------- FEDERATED ADAGRAD AGGREGATION --------------------
def fedadagrad(global_weights, client_weight_list, client_grads, accumulated_grads, learning_rate=1.0, epsilon=1e-6):
    for i in range(len(global_weights)):
        grad = np.average(client_grads[i], axis=0, weights=client_weight_list)
        accumulated_grads[i] += grad ** 2
        adaptive_update = learning_rate * grad / (np.sqrt(accumulated_grads[i]) + epsilon)
        global_weights[i] -= adaptive_update
    return global_weights

# -------------------- MAIN FEDERATED TRAINING LOOP --------------------
images, masks = load_data()
images, masks = shuffle(images, masks, random_state=42)

client1_count, client2_count, client3_count = 171, 305, 2588
client1_images, client1_masks = images[:client1_count], masks[:client1_count]
client2_images, client2_masks = images[client1_count:client1_count + client2_count], masks[client1_count:client1_count + client2_count]
client3_images, client3_masks = images[client1_count + client2_count:], masks[client1_count + client2_count:]

client_datasets = []
for imgs, mks in [(client1_images, client1_masks), (client2_images, client2_masks), (client3_images, client3_masks)]:
    x_train, x_test, y_train, y_test = train_test_split(imgs, mks, test_size=0.3, random_state=42)
    client_datasets.append((x_train, y_train, x_test, y_test))

global_model = build_model()
num_rounds, num_epochs = 10, 15
round_dice, round_iou = [], []

# Initialize accumulated gradients to zero
accumulated_grads = [np.zeros_like(w) for w in global_model.get_weights()]

for round_num in range(num_rounds):
    print(f"\n🔵 Round {round_num + 1}:")
    client_grads = [[] for _ in range(len(global_model.get_weights()))]
    weights_list = []
    avg_dice, avg_iou = [], []

    for (x_train, y_train, x_test, y_test) in client_datasets:
        client_model = build_model()
        client_model.set_weights(global_model.get_weights())

        client_model.fit(x_train, y_train, epochs=num_epochs, batch_size=16, verbose=0,
                         callbacks=[EarlyStopping(patience=3, restore_best_weights=True),
                                    ReduceLROnPlateau(patience=2)])

        updated_weights = client_model.get_weights()
        original_weights = global_model.get_weights()
        grads = [updated - original for updated, original in zip(updated_weights, original_weights)]

        for i in range(len(grads)):
            client_grads[i].append(grads[i])

        weights_list.append(len(x_train))

        loss, acc, miou, dice = client_model.evaluate(x_test, y_test, verbose=0)
        avg_dice.append(dice)
        avg_iou.append(miou)
        print(f"Client ({len(x_train)} samples): Dice: {dice:.4f}, IoU: {miou:.4f}")

    normalized_weights = [w / sum(weights_list) for w in weights_list]
    global_weights = global_model.get_weights()
    updated_weights = fedadagrad(global_weights, normalized_weights, client_grads, accumulated_grads)
    global_model.set_weights(updated_weights)

    round_dice.append(np.mean(avg_dice))
    round_iou.append(np.mean(avg_iou))

# -------------------- PERFORMANCE PLOT --------------------
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_rounds+1), round_dice, label="Dice Score", marker="o")
plt.plot(range(1, num_rounds+1), round_iou, label="IoU", marker="s")
plt.xlabel("Round")
plt.ylabel("Score")
plt.title("Performance Over Rounds (FedAdagrad)")
plt.legend()
plt.show()



I0000 00:00:1747196405.205988     100 service.cc:148] XLA service 0x7f86c8002dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747196405.206753     100 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747196405.206772     100 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747196406.622485     100 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747196438.223929     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
E0000 00:00:1747196460.003905     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1747196460.284196     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, pl

In [6]:
# -------------------- GLOBAL MODEL TESTING (METRICS ONLY) --------------------

def evaluate_global_model(model, test_images, test_masks):
    loss, acc, miou, dice = model.evaluate(test_images, test_masks, verbose=1)
    print("\n📊 Global Model Evaluation:")
    print(f"Loss      : {loss:.4f}")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Mean IoU  : {miou:.4f}")
    print(f"Dice Score: {dice:.4f}")

# -------------------- CHOOSE TEST DATA --------------------
# Combined test data from all clients
all_test_images = np.concatenate([client[2] for client in client_datasets])
all_test_masks  = np.concatenate([client[3] for client in client_datasets])

# -------------------- EVALUATE --------------------
evaluate_global_model(global_model, all_test_images, all_test_masks)



29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 293ms/step - accuracy: 0.9936 - dice_coefficient: 0.7978 - loss: 0.2873 - mean_io_u: 0.7256

📊 Global Model Evaluation:
Loss      : 0.2309
Accuracy  : 0.9899
Mean IoU  : 0.712
Dice Score: 0.792


